## Experiment 1
<br>
-- Numbers data with eye-blink removed vs Raw numbers data


In [1]:
import json
import utils
from data_handler import plot_images

import numpy as np

from collections import defaultdict

from IPython.display import clear_output

### Data Load

In [2]:
data_path = '/home/donghyun/eye_writing_classification/v2_dataset/200_points_dataset/'

with open(data_path + 'eog_raw_numbers_200.json') as f:
  eog_raw_numbers = json.load(f)

with open(data_path + 'eog_eb_remove_numbers_200.json') as f:
  eog_eb_remove_numbers = json.load(f)

with open(data_path + 'reference_data_200.json') as f:
  reference_data = json.load(f)

### Experiment

In [ ]:
# 10 times experiments with raw numbers and eye-blink removed numbers data using HybridBaseModel

class Config:
    split_ratio = 0.3
    ref_key = 'numbers'
    batch_size = 16               # fix : must be equaled with number of test pairs 
    n_batch = 50
    model_type = 'HybridBaseModel'
    epochs = 1000

times = 10

raw_numbers_dict = defaultdict(list)
ebr_numbers_dict = defaultdict(list)
for t in range(times):
    cfg = Config()
    
    _, raw_train_acc, raw_train_loss, raw_test_acc = utils.experiment(cfg, eog_raw_numbers, reference_data)
    _, ebr_train_acc, ebr_train_loss, ebr_test_acc = utils.experiment(cfg, eog_eb_remove_numbers, reference_data)

    raw_numbers_dict[t] = [raw_train_acc, raw_train_loss, raw_test_acc]
    ebr_numbers_dict[t] = [ebr_train_acc, ebr_train_loss, ebr_test_acc]

clear_output()

### Save

In [ ]:
import json

save_path = '/home/donghyun/eye_writing_classification/experiments/save/'

with open(save_path + 'experiment1_raw_numbers_results.json', 'w') as f:
    json.dump(dict(raw_numbers_dict),f)

with open(save_path + 'experiment1_eb_removed_numbers_results.json', 'w') as f:
    json.dump(dict(ebr_numbers_dict),f)

### Visualization

In [ ]:
save_path = '/home/donghyun/eye_writing_classification/experiments/save/'

ex1_numbers_results = json.dumps(save_path + 'experiment1_raw_numbers_results.json')
ex2_numbers_results = json.dumps(save_path + 'experiment1_eb_removed_numbers_results.json')